# Implement a SageMaker Multi-Model Endpoint for TensorFlow Vision models on a Triton Server from NVIDIA

Amazon SageMaker Multi-Model Endpoint (MME) is a cost-effective way of running multiple models behind a single endpoint. SageMaker manages the process of loading the target model into memory when needed, which leads to better utilization of the container resources and reduces cost. 

Multi-model endpoints are ideal when you have infrequently used models that can handle minor delays introduced by an occasional cold start. 

NVIDIA Triton Inference Server is an open source software that provides high performance inference on a wide variety of CPU and GPU hardware and supports all the major ML frameworks. It has many built-in features to improve inference throughput and achieves better utilization of the resources.  

Now the NVIDIA Triton Inference Server can be deployed on GPU based SageMaker ML instances. It supports the SageMaker MME API to for dynamic loading and unloading of models for implementing SageMaker multi-model endpoints. 

This notebook shows how to deploy multiple TensorFlow models trained on the MNIST dataset to a SageMaker MME using the NVIDIA Triton Server.

Here we use two different instances of an existing model artifact. The model used here was pre-trained on the MNIST dataset. If you want to learn how to train the model, please See [TensorFlow script mode training and serving](https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-python-sdk/tensorflow_script_mode_training_and_serving/tensorflow_script_mode_training_and_serving.ipynb). 

## Contents
1. [Introduction to NVIDIA Triton Server](#section1)
1. [Set up the environment](#section2)
1. [Transform TensorFlow Model structure](#section3)
  1. [Inspect the model using the saved_model_cli](#section3a)
  1. [Create the model configuration file](#section3b)
  1. [Create the tar ball in the required Triton structure](#section3c)
  1. [Create another copy of model artifacts for the MME](#section3d)
  1. [upload both model artifacts to S3](#section3e)
1. [Deploy model to SageMaker Multi-model Endpoint](#section4)
1. [Test the SageMaker Multi-model Endpoint](#section5)
1. [Clean up](#section6)

<a id='section1'></a>

## Introduction to NVIDIA Triton Server

[NVIDIA Triton Inference Server](https://github.com/triton-inference-server/server/) was developed specifically to enable scalable, cost-effective, and easy deployment of models in production. NVIDIA Triton Inference Server is open-source inference serving software that simplifies the inference serving process and provides high inference performance.

Some key features of Triton are:
* **Support for Multiple frameworks**: Triton can be used to deploy models from all major frameworks. Triton supports TensorFlow, ONNX, PyTorch, and many other model formats. 
* **Model pipelines**: Triton model ensemble represents a pipeline of one or more models or pre- / post-processing logic and the connection of input and output tensors between them. A single inference request to an ensemble will trigger the execution of the entire pipeline.
* **Concurrent model execution**: Multiple models (or multiple instances of the same model) can run simultaneously on the same GPU or on multiple GPUs for different model management needs.
* **Dynamic batching**: For models that support batching, Triton has multiple built-in scheduling and batching algorithms that combine individual inference requests together to improve inference throughput. These scheduling and batching decisions are transparent to the client requesting inference.
* **Diverse CPUs and GPUs**: The models can be executed on CPUs or GPUs for maximum flexibility and to support heterogeneous computing requirements.



<a id='section2'></a>

## Set up the environment

This notebook uses the Python 3 (Data Science) kernel. 



#### Install TensorFlow. This notebook is tested with version 2.11.

In [2]:
import sys

!{sys.executable} -m pip install "tensorflow>=2.1,<2.12"

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
# imports

import boto3, json, sagemaker, time
from sagemaker import get_execution_role
import numpy as np
from PIL import Image

#### For this exercise we download a TensorFlow model pre-trained on the MNIST data set from an Amazon S3 bucket. The model artifact is saved locally.

In [4]:
saved_model = "s3://sagemaker-sample-files/datasets/image/MNIST/model/tensorflow-training-2020-11-20-23-57-13-077/model.tar.gz"
!aws s3 cp $saved_model models/SavedModel/

download: s3://sagemaker-sample-files/datasets/image/MNIST/model/tensorflow-training-2020-11-20-23-57-13-077/model.tar.gz to models/SavedModel/model.tar.gz


#### You should have already configured the default IAM role for running this notebook with access to the model artifacts and the NVIDIA Triton Server image in Amazon Elastic Container Registry (ECR).

In [5]:
sm_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket_name = sm_session.default_bucket()
region = boto3.Session().region_name

print(f"Default IAM Role: {role}")
print(f"Default S3 Bucket: {bucket_name}")
print(f"AWS Region: {region}")

Default IAM Role: arn:aws:iam::095351214964:role/service-role/AmazonSageMaker-ExecutionRole-20200130T133110
Default S3 Bucket: sagemaker-us-east-1-095351214964
AWS Region: us-east-1


#### Download the Triton Server image from Amazon ECR.

In [6]:
# Amazon ECR images are region specific

account_id_map = {
    "us-east-1": "785573368785",
    "us-east-2": "007439368137",
    "us-west-1": "710691900526",
    "us-west-2": "301217895009",
    "eu-west-1": "802834080501",
    "eu-west-2": "205493899709",
    "eu-west-3": "254080097072",
    "eu-north-1": "601324751636",
    "eu-south-1": "966458181534",
    "eu-central-1": "746233611703",
    "ap-east-1": "110948597952",
    "ap-south-1": "763008648453",
    "ap-northeast-1": "941853720454",
    "ap-northeast-2": "151534178276",
    "ap-southeast-1": "324986816169",
    "ap-southeast-2": "355873309152",
    "cn-northwest-1": "474822919863",
    "cn-north-1": "472730292857",
    "sa-east-1": "756306329178",
    "ca-central-1": "464438896020",
    "me-south-1": "836785723513",
    "af-south-1": "774647643957",
}

if region not in account_id_map.keys():
    raise ("UNSUPPORTED REGION")

base = "amazonaws.com.cn" if region.startswith("cn-") else "amazonaws.com"

mme_triton_image_uri = (
    "{account_id}.dkr.ecr.{region}.{base}/sagemaker-tritonserver:22.07-py3".format(
        account_id=account_id_map[region], region=region, base=base
    )
)

print(f"Triton server image: {mme_triton_image_uri}")

Triton server image: 785573368785.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tritonserver:22.07-py3


In [7]:
# Extract the model into a local folder

!tar -xf models/SavedModel/model.tar.gz -C models/SavedModel/ --no-same-owner

<a id='section3'></a>

## Transform TensorFlow Model structure


The model that we want to deploy currently has the following structure:

```
00000000
        ├── saved_model.pb
        ├── assets/
        └── variables/
            ├── variables.data-00000-of-00001
            └── variables.index
```
For Triton, the model needs to have the following structure:
```
<model-name>
├── config.pbtxt
└── 1
    └── model.savedmodel
        ├── saved_model.pb
        ├── assets/
        └── variables/
            ├── variables.data-00000-of-00001
            └── variables.index
            

```

In [8]:
prefix = "triton-mme"

! mkdir -p models/$prefix/MNIST1/1
! cp models/SavedModel/00000000 --recursive ./models/$prefix/MNIST1/1/model.savedmodel/

<a id='section3a'></a>

### Inspect the model using the "saved_model_cli" command.

In order to create the `config.pbtxt` we need to confirm the model inputs and outputs (Signature).
We use the "saved_model_cli" to inspect the model and take note of the input and output shape.

In [9]:
!saved_model_cli show --all --dir  {"models/SavedModel/00000000"}

2023-03-12 17:08:59.418236: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-12 17:08:59.980340: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-12 17:08:59.980382: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-12 17:09:01.543653: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

<a id='section3b'></a>

### Create the `config.pbtxt` file

Triton requires a [Model Configuration file](https://github.com/triton-inference-server/server/blob/main/docs/model_configuration.md) called `config.pbtxt`. 

We create one below in the local folder for uploading with the model artifact.


In [10]:
%%writefile models/triton-mme/MNIST1/config.pbtxt
platform: "tensorflow_savedmodel"
max_batch_size: 0

instance_group {
  count: 1
  kind: KIND_GPU
}

dynamic_batching {

}

input [
  {
    name: "input_1"
    data_type: TYPE_FP32
    dims: [-1, 28, 28, 1]
  }
]
output [
  {
    name: "output_1"
    data_type: TYPE_FP32
    dims: [-1, 10]
  }
]

Writing models/triton-mme/MNIST1/config.pbtxt


<a id='section3c'></a>

### Create a tar ball of the first model in the required folder structure for Triton

In [11]:
!tar -C models/triton-mme -czvf models/triton-mme/TritonModel1.tar.gz MNIST1/

MNIST1/
MNIST1/config.pbtxt
MNIST1/1/
MNIST1/1/model.savedmodel/
MNIST1/1/model.savedmodel/variables/
MNIST1/1/model.savedmodel/variables/variables.data-00000-of-00001
MNIST1/1/model.savedmodel/variables/variables.index
MNIST1/1/model.savedmodel/saved_model.pb
MNIST1/1/model.savedmodel/assets/


<a id='section3d'></a>

### Create a second copy of the model artifacts under a different name for the Multi-Model Endpoint.

In [12]:
# Save another copy of the model
!mkdir -p models/triton-mme/MNIST2/1

# cp ./models/triton-mme/MNIST1/* --recursive ./models/triton-mme/MNIST2
!cp ./models/triton-mme/MNIST1/* --recursive ./models/triton-mme/MNIST2

!tar -C models/triton-mme -czvf models/triton-mme/TritonModel2.tar.gz MNIST2/

MNIST2/
MNIST2/config.pbtxt
MNIST2/1/
MNIST2/1/model.savedmodel/
MNIST2/1/model.savedmodel/variables/
MNIST2/1/model.savedmodel/variables/variables.data-00000-of-00001
MNIST2/1/model.savedmodel/variables/variables.index
MNIST2/1/model.savedmodel/saved_model.pb
MNIST2/1/model.savedmodel/assets/


<a id='section3e'></a>

### Upload model artifacts for both models to Amazon S3

In [13]:
tf_models_mnist = ["TritonModel1.tar.gz", "TritonModel2.tar.gz"]


model_1_location = f"s3://{bucket_name}/{prefix}/{tf_models_mnist[0]}"
model_2_location = f"s3://{bucket_name}/{prefix}/{tf_models_mnist[1]}"

In [14]:
!aws s3 cp models/$prefix/TritonModel1.tar.gz $model_1_location
!aws s3 cp models/$prefix/TritonModel2.tar.gz $model_2_location

upload: models/triton-mme/TritonModel1.tar.gz to s3://sagemaker-us-east-1-095351214964/triton-mme/TritonModel1.tar.gz
upload: models/triton-mme/TritonModel2.tar.gz to s3://sagemaker-us-east-1-095351214964/triton-mme/TritonModel2.tar.gz


<a id='section4'></a>

## Deploy the TensorFlow models to a Multi-Model Endpoint


### Define the serving container

In the container definition below, we need to pass in the following parameters.
- Image: Triton server image URI that supports deploying multi-model endpoints with GPUs.
- URI to S3 folder that contains all the models that SageMaker multi-model endpoint will use to load and serve predictions. 
- Mode: Set to MultiModel   


In [15]:
model_data_url = f"s3://{bucket_name}/{prefix}/"

container = {"Image": mme_triton_image_uri, "ModelDataUrl": model_data_url, "Mode": "MultiModel"}


### Create a model object using the container defined above

Create the model object using the Boto3 create_model API. We pass the container definition to the create model API along with the model name and execution role.


In [16]:
ts = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

sm_model_name = f"{prefix}-mdl-{ts}"

sm_client = boto3.client(service_name="sagemaker")

create_model_response = sm_client.create_model(
    ModelName=sm_model_name, ExecutionRoleArn=role, PrimaryContainer=container
)

print("Model Arn: " + create_model_response["ModelArn"])

Model Arn: arn:aws:sagemaker:us-east-1:095351214964:model/triton-mme-mdl-2023-03-12-17-09-09


## Deploy and test the multi-model endpoint

Create a multi-model endpoint configurations using the create_endpoint_config Boto3 API. 
We specify an accelerated GPU computing instance as the instance type. For testing we specify a single instance. In real scenarios we recommend the value of initial instance count to be two or higher for high availability.  

### Create endpoint configuration

In [17]:
endpoint_config_name = f"{prefix}-epc-{ts}"

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.g4dn.xlarge",
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": sm_model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

Endpoint Config Arn: arn:aws:sagemaker:us-east-1:095351214964:endpoint-config/triton-mme-epc-2023-03-12-17-09-09



### Create Multi-Model Endpoint

Using the above endpoint configuration we create a new SageMaker endpoint and wait for the deployment to finish. The status will change to *In Service* once the deployment is successful.



In [18]:
endpoint_name = f"{prefix}-ep-{ts}"

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-east-1:095351214964:endpoint/triton-mme-ep-2023-03-12-17-09-09


In [19]:
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-1:095351214964:endpoint/triton-mme-ep-2023-03-12-17-09-09
Status: InService


<a id='section5'></a>

## Invoke target models behind the Multi-Model Endpoint

Once the endpoint is successfully created, we can send inference requests to the multi-model endpoint using invoke_endpoint API. We specify the target model in the invocation call and pass in the payload for each model type.

### Let's download some test data


In [20]:
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_data.npy train_data.npy
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_labels.npy train_labels.npy

runtime_sm_client = boto3.client("sagemaker-runtime")


train_data = np.load("train_data.npy")
train_data = train_data.reshape((train_data.shape[0], 28, 28, 1))

train_labels = np.load("train_labels.npy")

download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/train_data.npy to ./train_data.npy
download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/train_labels.npy to ./train_labels.npy


In [21]:
%%time

for tf_model in tf_models_mnist:  # iterate through the two models
    print(f"\nModel invoked: {tf_model}")

    for i in range(10):
        input_data = train_data[0].tolist()

        payload = {
            "inputs": [
                {
                    "name": "input_1",
                    "shape": [1, 28, 28, 1],
                    "datatype": "FP32",
                    "data": train_data[i].tolist(),
                }
            ]
        }

        response = runtime_sm_client.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType="application/octet-stream",
            Body=json.dumps(payload),
            TargetModel=tf_model,
        )

        predictions = json.loads(response["Body"].read())["outputs"][0]["data"]
        predictions = np.array(predictions, dtype=np.float32)
        predictions = np.argmax(predictions)
        print(f"Predicted value: {predictions},\tActual value: {train_labels[i]}")


Model invoked: TritonModel1.tar.gz
Predicted value: 7,	Actual value: 7
Predicted value: 3,	Actual value: 3
Predicted value: 4,	Actual value: 4
Predicted value: 6,	Actual value: 6
Predicted value: 1,	Actual value: 1
Predicted value: 8,	Actual value: 8
Predicted value: 1,	Actual value: 1
Predicted value: 0,	Actual value: 0
Predicted value: 9,	Actual value: 9
Predicted value: 8,	Actual value: 8

Model invoked: TritonModel2.tar.gz
Predicted value: 7,	Actual value: 7
Predicted value: 3,	Actual value: 3
Predicted value: 4,	Actual value: 4
Predicted value: 6,	Actual value: 6
Predicted value: 1,	Actual value: 1
Predicted value: 8,	Actual value: 8
Predicted value: 1,	Actual value: 1
Predicted value: 0,	Actual value: 0
Predicted value: 9,	Actual value: 9
Predicted value: 8,	Actual value: 8
CPU times: user 97.3 ms, sys: 12.2 ms, total: 110 ms
Wall time: 3.84 s


<a id='section6'></a>

## Clean up
We strongly recommend deleting the real-time endpoint created to stop incurring cost when finished with the example.

In [22]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=sm_model_name)

print(f"Deleted Endpoint: {endpoint_name}")
print(f"Deleted Endpoint Config: {endpoint_config_name}")
print(f"Deleted Model: {sm_model_name}")

Deleted Endpoint: triton-mme-ep-2023-03-12-17-09-09
Deleted Endpoint Config: triton-mme-epc-2023-03-12-17-09-09
Deleted Model: triton-mme-mdl-2023-03-12-17-09-09
